In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/damoncrockett/ivpytest/src')
from ivpy import *
from ivpy.extract import extract,norm
from ivpy.utils import resize
import glob,os
import warnings
warnings.filterwarnings('ignore')

We want to keep the sort order the same as for the other models.

In [3]:
f = '/Users/damoncrockett/Desktop/textureparams2.csv'
xf = pd.read_csv(f,header=None)

In [4]:
DIR = '/Users/damoncrockett/simspace/src/assets/json/img/'

In [5]:
# won't be used in the web app, but useful for futzing with the images prior to

xf['localpath'] = [DIR + item + '.jpeg' for item in xf[0]]

### Creating localhost paths

In [6]:
xf['localhostpath'] = ['http://localhost:8888/img/' + os.path.basename(item) for item in xf.localpath]

### Random model

In [7]:
X = pd.DataFrame(index=xf.index)

In [8]:
for i in range(3):
    X[i] = np.random.random_sample(len(xf))

### Metadata

In [9]:
xf['edition'] = [item.split('_')[0] for item in xf[0]]
xf['leaf'] = ['_'.join(item.split('_')[:-1]) for item in xf[0]]
xf.rename(columns={0:'fullname'},inplace=True)

### Creation of simspaces

The target problem is visualizing the similarity spaces of different texture models. In order to visualize these spaces, we need dimension reduction (to 2 dimensions). For now, we'll just use t-SNE and UMAP, since they are different simspaces and can return 2-dimensional display coordinates.

In [10]:
from ivpy.reduce import pca,tsne,umap

In [11]:
xf[['xp','yp']] = norm(pca(X,n_components=2))
xf[['xt','yt']] = norm(tsne(X))
xf[['xu','yu']] = norm(umap(X))

### Adding in cluster ids from SP model

In [12]:
cf = pd.read_csv('/Users/damoncrockett/Desktop/clusterNums.csv')
xf['clusterNum'] = cf.clusterNum

### JSON creation

In [13]:
import json

In [14]:
def row2jso(xf,xcol,ycol,i):
    row = xf.loc[i]
    jso = {}
    jso['imgpath'] = row.localhostpath
    jso['x'] = row[xcol]
    jso['y'] = row[ycol]
    jso['edition'] = row.edition
    jso['leaf'] = row.leaf
    jso['fullname'] = row.fullname
    jso['clusterNum'] = int(row.clusterNum)

    return jso

In [15]:
pcaArray = [row2jso(xf,'xp','yp',i) for i in xf.index]
tsneArray = [row2jso(xf,'xt','yt',i) for i in xf.index]
umapArray = [row2jso(xf,'xu','yu',i) for i in xf.index]

In [16]:
JSONDIR = '/Users/damoncrockett/simspace/src/assets/json/'

In [17]:
with open(JSONDIR+"random4_pca.json",'w') as outfile:
    json.dump(pcaArray,outfile)

with open(JSONDIR+"random4_tsne.json",'w') as outfile:
    json.dump(tsneArray,outfile)

with open(JSONDIR+"random4_umap.json",'w') as outfile:
    json.dump(umapArray,outfile)